In [1]:
import pickle
from collections import OrderedDict
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [2]:
from exfor_tools.distribution import AngularDistribution

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
from jitr.optical_potentials import kduq, wlh

In [4]:
import elm

In [5]:
neutron = (1, 0)
proton = (1, 1)

In [6]:
with open(Path("../prior/prior_distribution.pickle"), "rb") as f:
    prior_distribution = pickle.load(f)
param_samples = elm.elm.array_to_list(prior_distribution.rvs(1000))

## read in measurements and pre-computed workspaces

In [7]:
corpus_dir = Path("./corpus/")
output_dir = Path("./elm_prior_propagation/")
output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
with open(corpus_dir / "nn_elastic_data.pkl", "rb") as f:
    nn_elastic_data = pickle.load(f)
with open(corpus_dir / "pp_elastic_data.pkl", "rb") as f:
    pp_elastic_data = pickle.load(f)

In [9]:
with open(corpus_dir / "nn_elastic_workspaces.pkl", "rb") as f:
    nn_elastic_workspaces = pickle.load(f)
with open(corpus_dir / "pp_elastic_workspaces.pkl", "rb") as f:
    pp_elastic_workspaces = pickle.load(f)

In [10]:
kduq_params_nn = kduq.get_samples_federal(neutron)
kduq_params_pp = kduq.get_samples_federal(proton)

## set up corpora from workspaces

In [11]:
nn_corpus = elm.corpus.ElasticAngularCorpus(
    elm.elm.calculate_elastic_differential_xs,
    "ELM",
    elm.elm.NUM_PARAMS,
    "dXS/dA",
    nn_elastic_workspaces,
    nn_elastic_data,
)
pp_corpus = elm.corpus.ElasticAngularCorpus(
    elm.elm.calculate_elastic_differential_xs,
    "ELM",
    elm.elm.NUM_PARAMS,
    "dXS/dRuth",
    pp_elastic_workspaces,
    pp_elastic_data,
)
with open(output_dir / "elm_nn_corpus.pkl", "wb") as f:
    pickle.dump(nn_corpus, f)
with open(output_dir / "elm_pp_corpus.pkl", "wb") as f:
    pickle.dump(pp_corpus, f)

## Propagate ELM prior into constraint observables

In [12]:
yth_nn = []
for i, constraint in enumerate(tqdm(nn_corpus.constraints)):
    yth_nn.append(np.vstack([constraint.model(p) for p in param_samples]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [02:21<00:00,  2.17s/it]


In [13]:
yth_pp = []
for i, constraint in enumerate(tqdm(pp_corpus.constraints)):
    yth_pp.append(np.vstack([constraint.model(p) for p in param_samples]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [05:27<00:00,  2.05s/it]


## Write posterior predictive distributions to disk

In [14]:
with open(output_dir / "nn_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [15]:
with open(output_dir / "pp_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)